# General Workflow:
    1. Downloaded the vessel traffic data as .csv file from the https://marinecadastre.gov/accessais/ website
    2. Import/Read the csv file in jupyter notebook using it as a data table for input
    3. Mapping the Lat-Long over Open Street Map for different vessel type using Bounding Box location
    4. Adding Sattellite image from USGS as the base layer
    5. Counting the vessle frequency based on the IMO Code**

# 1. CSV file Reading

**The data downloaded here is for one single (Date: 11/01/2022)**

In [53]:
import pandas as pd

file_path = r'C:\Users\Redwan Kabir\Downloads\AIS_170398259956363288_6194-1703982600146.csv'

# Use pandas to read the CSV file
data = pd.read_csv(file_path)

# Display the data or perform operations as needed
print(data.head())  # Displays the first few rows of the CSV file


        MMSI         BaseDateTime       LAT        LON   SOG    COG  Heading  \
0  367425230  2022-11-01T00:00:00  37.78147 -122.38650   0.0   90.1      511   
1  368060690  2022-11-01T00:00:00  37.91166 -122.34869   0.0  252.0      511   
2  366963980  2022-11-01T00:00:00  37.85361 -122.47401  13.9  118.0      117   
3  367469070  2022-11-01T00:00:00  37.86830 -122.31493   0.0  319.8      511   
4  367301480  2022-11-01T00:00:01  37.89246 -122.43417  10.1  149.9      151   

          VesselName         IMO CallSign  VesselType  Status  Length  Width  \
0  CALIFORNIA SPIRIT         NaN  WDF2131        60.0     0.0    27.0    7.0   
1      GREEN ROSETTA         NaN  WDK5002        36.0     NaN    14.0    5.0   
2      SAN FRANCISCO         NaN  WZC3886        60.0     0.0    46.0    9.0   
3  SUNSET HORNBLOWER         NaN  WDF5955        60.0     0.0    29.0    7.0   
4              VALOR  IMO9409924  WDD8691        31.0     0.0    30.0   12.0   

   Draft  Cargo TransceiverClass  
0  

# 2. Mapping Vessle based on IMO Code using my AOI Bounding Box

**For example here I used IMO code: 'IMO9550773' for plotting over the open street map**

In [45]:
import pandas as pd
import folium

# Read the CSV file into a DataFrame
file_path = r'C:\Users\Redwan Kabir\Downloads\AIS_170398259956363288_6194-1703982600146.csv'
data = pd.read_csv(file_path)

# Filter data for the specific vessel IMO9550773
filtered_data = data[data['IMO'] == 'IMO9550773']

# Create a base map centered at a specific location
map_osm = folium.Map(location=[37.85, -122.4], zoom_start=12)  # Set the initial center and zoom level

# Define colors for different vessel types (you can modify/add colors as needed)
colors = {
    'Cargo': 'blue',
    'Passenger': 'green',
    'Tanker': 'red',
    # Add more types or categories as required
}

# Add vessel locations to the map with different colored markers based on vessel type
for index, row in filtered_data.iterrows():
    # Extract latitude, longitude, and vessel type
    lat = row['LAT']
    lon = row['LON']
    vessel_type = row['VesselType']
    
    # Determine the color for the marker based on vessel type
    marker_color = colors.get(vessel_type, 'gray')  # Default to gray if vessel type not in colors dictionary
    
    # Add markers for vessel locations with different colors based on vessel type
    folium.CircleMarker(
        location=[lat, lon],
        radius=5,
        color=marker_color,
        fill=True,
        fill_color=marker_color,
        popup=row['VesselName']
    ).add_to(map_osm)

# Create legend for different vessel types
for vessel_type, color in colors.items():
    legend_html = f'<i style="background:{color}"></i> {vessel_type}'
    folium.Marker(
        [0, 0],
        icon=folium.DivIcon(html=legend_html),
        popup=vessel_type
    ).add_to(map_osm)

# Display the map with legend
map_osm


# 3. Adding multiple Vessle types (IMO) in the same image for geotagging

**Here I used three vessle types in this map for visualization purpose over the sattelite image. The legends are also marked using different colors**

In [54]:
import pandas as pd
import folium

# Reading CSV file into a DataFrame
file_path = r'C:\Users\Redwan Kabir\Downloads\AIS_170398259956363288_6194-1703982600146.csv'
data = pd.read_csv(file_path)

# Filter data for the specific vessels
filtered_data = data[data['IMO'].isin(['IMO9550773', 'IMO9300207', 'IMO9164275'])]

# Creating a base map centered at a specific location
map_osm = folium.Map(location=[37.85, -122.4], zoom_start=12, control_scale=True)  # Set the initial center and zoom level

# Add USGS satellite imagery as a base layer
folium.TileLayer(
    tiles='https://basemap.nationalmap.gov/arcgis/rest/services/USGSImageryOnly/MapServer/tile/{z}/{y}/{x}',
    attr='USGS',
    name='USGS Imagery',
    overlay=True,
).add_to(map_osm)


# colors for different vessels
colors = {
    'IMO9550773': 'blue',
    'IMO9300207': 'yellow',
    'IMO9164275': 'red',
}


# Creating legend for different vessels with their names and background color
legend_html = '''
     <div style="position: fixed; 
     bottom: 50px; left: 50px; width: 200px; height: auto; 
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color: #fff; padding: 10px;
     ">&nbsp; <b>Legend</b> <br>'''

for imo_number, color in colors.items():
    vessel_name = data[data['IMO'] == imo_number]['VesselName'].iloc[0]  # Get vessel name
    legend_html += f'&nbsp; <i class="fa fa-circle" style="color:{color}"></i> {imo_number} - {vessel_name} <br>'
    
legend_html += '</div>'
map_osm.get_root().html.add_child(folium.Element(legend_html))

# vessel locations to the map with different colored markers based on the vessels
for index, row in filtered_data.iterrows():
    # Extract latitude, longitude, and IMO number
    lat = row['LAT']
    lon = row['LON']
    imo_number = row['IMO']
    
    # the color for the marker based on the IMO number
    marker_color = colors.get(imo_number, 'gray')  # Default to gray if IMO number not in colors dictionary
    
    # markers for vessel locations with different colors based on the IMO number
    folium.CircleMarker(
        location=[lat, lon],
        radius=5,
        color=marker_color,
        fill=True,
        fill_color=marker_color,
        popup=row['VesselName']
    ).add_to(map_osm)

# Display the map
map_osm


# 4. Vessle Type count on that particular day

In [34]:
import pandas as pd

# Reading the CSV file into a DataFrame
file_path = r'C:\Users\Redwan Kabir\Downloads\AIS_170398259956363288_6194-1703982600146.csv'
data = pd.read_csv(file_path)

# Filter data for the specific vessels IMO9550773 and IMO9604122
filtered_data = data[data['IMO'].isin(['IMO9550773', 'IMO9300207', 'IMO9164275'])]

# Count occurrences of each IMO code
imo_counts = filtered_data['IMO'].value_counts()

# Display the count of each IMO code
print(imo_counts)


IMO9300207    478
IMO9164275    477
IMO9550773    437
Name: IMO, dtype: int64
